## Selenium으로 브라우저에서 자동으로 클릭, 검색, 스크롤 등을 작동하며, 정보 크롤링

In [ ]:
#step0) 브라우저의 조작을 컴퓨터가 자동으로 할 수 있게 해주는 라이브러리 설치
!pip install selenium
#문법이 과거 버전일 경우에도 잘 적용시켜주기 Update 쓴 것도 추가하기  
!pip install -U selenium

In [ ]:
# #step0) 브라우저의 조작을 컴퓨터가 자동으로 할 수 있게 해주는 라이브러리 설치
# !pip install selenium
# #문법이 과거 버전일 경우에도 잘 적용시켜주기 Update 쓴 것도 추가하기  
# !pip install -U selenium
#step1) 브라우저 on/off 하는 , 컴퓨터용 키보드를 만드는 라이브러리 가져오기 
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
#step1) 선택자(id 냐 class냐 css selector냐 등을 구분하는)의 구분자 역할을 하는 라이브러리 불러오기
from selenium.webdriver.common.by import By
#step2) 웹브라우저 시작 
driver=wb.Chrome()
driver.get("http://www.naver.com")
#step3)검색창에 아시아게임 입력 후 엔터
#위에서 시작한 웹브라우저의 개발자도구에서 '검색창" ID찾기  
search=driver.find_element(By.ID,"query")
search.send_keys("아시아게임")
search.send_keys(Keys.ENTER)
#step4) 자동으로 스크롤 최하단으로 이동 
body=driver.find_element(By.TAG_NAME,"body")
body.send_keys(Keys.END)
#step5) 뒤로 가고 검색창 초기화하고 새로운 검색어 입력 후 클릭 
#주의)검색 다시 할 때 검색창 초기화하기
search=driver.find_element(By.ID,"query")
search.send_keys("이강인")
btn=driver.find_element(By.CSS_SELECTOR,'btn_search')
btn.click()
#step6)소속팀 글자정보 수집 
#개발자도구에서 요소 부분을 찾고, 우클릭, 요소 복사, 선택자 한번에 가져오되,
#내가 선택한 요소 딱 하나만 반환한다
#만약, 복수개를 수집하고 싶다면 nth-child나 id를 삭제한다. 
team=driver.find_element(By.CSS_SELECTOR,"#main_pack > section.sc_new.cs_common_module.case_normal._au_people_content_wrap.color_16._cs_sports_people > div.cm_content_wrap._content > div.cm_content_area._cm_content_area_profile > div > div.detail_info > dl > div:nth-child(3) > dd > a")
team.text
#stpe7)창을 종료
driver.close()


## 한솥도시락 웹브라우저에 접속 후 메뉴, 가격 정보 크롤링


In [1]:
# !pip install selenium
# !pip install -U selenium
#step0)브라우저 키고, 컴퓨터용 키보드 조작하는 라이브러리 
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
#선택자 구분자에 의해 크롤링할 수 있게 하는 라이브러리 
from selenium.webdriver.common.by import By
#컴퓨터가 하는 것이 발각되면 안되기에 쉬는 시간 추가하는 라이브러리 불러오기
import time
#step1)한솥페이지 열기
driver=wb.Chrome()
driver.get("https://www.hsd.co.kr/menu/menu_list")
time.sleep(1)
#주의)아래 step2처럼 콘텐츠만 가져올 때 웹브라우저에서 '더보기'버튼을 눌러야 
#전체 정보가 나올 수 있다

try:
    # 실행하고싶은 실행문장 작성
    for i in range(30):
        btn_more = driver.find_element(By.CLASS_NAME, 'c_05')
        btn_more.click()
        time.sleep(1) #1초
except:
    # 에러가 발생했을 경우 실행할 실행문장 작성~
    print("클릭 완료~")

#step2) 메뉴정보와 가격정보를 리스트형태로 가져오기,
#주의)nth 나 id가 있을 경우 지워야 여러 개의 리스트 형태 출력 가능하다! 
#오류search=driver.find_elements(By.CSS_SELECTOR,"#menuList_508 > div > div.item-text > h4")
search=driver.find_elements(By.CSS_SELECTOR,'h4.h.fz_03')
#구분자 By > CSS_SELECTOR로 사용할 때는 (태그명 선택자 구분자기호 구분자명) 작성 ! 
# driver.find_element(By.CLASS_NAME,'h.fz_03').text
price=driver.find_elements(By.CSS_SELECTOR," div > div.item-text > div > strong")

#step3)콘텐츠만 출력하는 text이용해서 깔끔한 리스트로 만들어주기
menu_list=[]
price_list=[]
for i in range(len(price)):
    menu_list.append(search[i].text)
    price_list.append(price[i].text)
    
#step4) 브라우저 종료
driver.quit()

#step5) 딕셔너리형태로 만든 후 데이터프레임화하기 
import pandas as pd 
dic = {'메뉴':menu_list, '가격':price_list}
data = pd.DataFrame(dic)

#step6) 파일 내보내기 
data.to_csv('data/한솥메뉴.csv')




    


클릭 완료~


## 네이버에서 동명동맛집 검색 후 이미지 수집하기 

In [2]:
# !pip install selenium
# !pip install -U selenium
#step0) 라이브러리 불러오기
from selenium import webdriver as wb #브라우저 키고끄는
from selenium.webdriver.common.keys import Keys #키 조작하는 
from selenium.webdriver.common.by import By #선택자 구분자 
import time #시간주는 
import os # Opreating system 폴더 생성 
from bs4 import BeautifulSoup as bs #html 형태로 객체화해주는 
from urllib.request import urlretrieve # 웹상의 파일(이미지, html페이지 등)을 다운로드하는 

#step1) os 라이브러리 폴더 생성 
#바탕화면에서 아무 폴더에서 링크 복사해오고 뒤에 파일명 적기 
if not os.path.isdir('C:\\Users\\gjaischool1\\Desktop\\동명동맛집'):
    print('폴더생성')
    os.mkdir('C:\\Users\\gjaischool1\\Desktop\\동명동맛집')
else:
    print('존재하는 이름의 폴더입니다.')
#step2)네이버 메인페이지 열고, 전체창으로, '동명동맛집' 키 입력
driver=wb.Chrome()
driver.get('https://www.naver.com/')
driver.maximize_window()
search=driver.find_element(By.ID, 'query')
search.send_keys('동명동맛집')
search.send_keys(Keys.ENTER)
#step3)이미지 탭 클릭 후 아래로 끝까지 스크롤
driver.find_element(By.CSS_SELECTOR,'#lnb > div.lnb_group > div > div.lnb_nav_area._nav_area_root > div > div.api_flicking_wrap._conveyer_root > div:nth-child(2) > a').click()
body = driver.find_element(By.CSS_SELECTOR,'body')
for i in range(20):
    body.send_keys(Keys.END)
    time.sleep(2)
time.sleep(10)
#step4) 이미지 태그(그 속의 src 속성) 수집 후 이미지 파일로 저장 
img=driver.find_elements(By.CSS_SELECTOR,'img._image._listImage')
img_list=[]
for i in img:
    img_list.append(i.get_attribute('src'))

# html로 객체화해서 src 가져올 수도 있다.
# #bs(페이지정보, 파싱방법)
# soup=bs(driver.page_source, 'lxml')
# img=soup.select('img._image._listImage')
# #반복문을 통해 각 이미지 태그에서 src 속성을 추출
# img_list=[]
# for i in img:
#     img_list.append(i['src'])

#step5) 이미지파일로 저장 
for i in range(100):
    urlretrieve(img_list[i],f'C:\\Users\\gjaischool1\\Desktop\\동명동맛집\\동명동맛집{i+1}.jpg')



존재하는 이름의 폴더입니다.
